# Mpox

In [4]:
import numpy as np
import torchvision.models as tvm

import torch.nn as nn
import torch
import random
from tqdm import tqdm
from torch.utils.data import DataLoader

import os

import torch.optim as to
import matplotlib.pyplot as plt




from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset

%run model.ipynb



cuda


In [ ]:
!unzip '/Monkeypox Skin Image Dataset-2'

In [ ]:
torch.cuda.empty_cache()

In [5]:
import os
import torch
import random
import numpy as np
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

# Set a random seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # For GPU reproducibility
random.seed(SEED)


def load_msid_data(data_dir, img_size=224):
    # Define the classes
    classes = ['Chickenpox', 'Measles', 'Monkeypox', 'Normal']
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
    print(class_to_idx)
    num_classes = len(classes)

    # Create lists to store paths and labels
    all_paths = []
    all_labels = []

    # Collect all image paths and labels
    for class_name in classes:
        class_dir = os.path.join(data_dir, class_name)
        class_idx = class_to_idx[class_name]
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(class_dir, img_name)
                all_paths.append(img_path)
                all_labels.append(class_idx)

    # Split the data while maintaining class proportions
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_paths, all_labels,
        test_size=0.2,
        stratify=all_labels,
        random_state=SEED  # Seed for reproducibility
    )

    # Define transform - only resize and convert to tensor, then divide by 255
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),  # ToTensor automatically divides by 255
    ])

    # Convert images to tensors
    def paths_to_tensor(image_paths):
        tensors = []
        for img_path in image_paths:
            image = Image.open(img_path).convert('RGB')
            tensor = transform(image)
            tensors.append(tensor)
        return torch.stack(tensors)

    # Create one-hot encoded labels
    def to_one_hot(labels, num_classes):
        labels_tensor = torch.tensor(labels)
        one_hot = torch.zeros(len(labels), num_classes)
        one_hot.scatter_(1, labels_tensor.unsqueeze(1), 1)
        return one_hot

    # Create tensors
    X_train = paths_to_tensor(train_paths)
    X_test = paths_to_tensor(test_paths)
    y_train = to_one_hot(train_labels, num_classes)
    y_test = to_one_hot(test_labels, num_classes)

    # Print dataset statistics
    print("\nDataset Statistics:")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"y_test shape: {y_test.shape}")

    print("\nClass distribution:")
    for cls in classes:
        train_count = y_train[:, class_to_idx[cls]].sum().item()
        test_count = y_test[:, class_to_idx[cls]].sum().item()
        print(f"{cls}: {train_count} train, {test_count} test")

    return X_train, y_train, X_test, y_test, classes


if __name__ == "__main__":
    data_dir = "Monkeypox Skin Image Dataset-2"
    X_train, y_train, X_test, y_test, classes = load_msid_data(data_dir)


{'Chickenpox': 0, 'Measles': 1, 'Monkeypox': 2, 'Normal': 3}

Dataset Statistics:
X_train shape: torch.Size([616, 3, 224, 224])
X_test shape: torch.Size([154, 3, 224, 224])
y_train shape: torch.Size([616, 4])
y_test shape: torch.Size([154, 4])

Class distribution:
Chickenpox: 86.0 train, 21.0 test
Measles: 73.0 train, 18.0 test
Monkeypox: 223.0 train, 56.0 test
Normal: 234.0 train, 59.0 test


In [6]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [7]:
import timm
print(timm.__version__)

1.0.12


In [ ]:
# np.save('X_train.npy', X_train)
# np.save('y_train.npy', y_train)
# np.save('X_test.npy', X_test)
# np.save('y_test.npy', y_test)


In [10]:
def evaluate(y_true, y_pred):
    """
    Evaluates the performance of a classification model using precision, recall, and F1-score.

    Parameters:
    y_true (list or array-like): True labels.
    y_pred (list or array-like): Predicted labels by the model.

    Returns:
    tuple: Precision, Recall, and F1-Score, calculated using macro averaging.
    """

    # Calculate precision score using macro averaging (treats all classes equally)
    precision = precision_score(y_true, y_pred, average='macro')

    # Calculate recall score using macro averaging
    recall = recall_score(y_true, y_pred, average='macro')

    # Calculate F1-score using macro averaging
    f1 = f1_score(y_true, y_pred, average='macro')

    # Return the calculated metrics as a tuple
    return precision, recall, f1


In [11]:


def replace_context_modules(model, Module, dilation):
    """
    Replaces the context_module in specific blocks of a model's fourth stage with a custom module.

    Parameters:
    model: The model containing stages and blocks where replacements are to be made.
    Module: Custom module to replace the existing context_module.
    seed: Random seed for reproducibility in the custom module.
    """

    # Access the fourth stage (index 3) of the model
    stage = model.stages[3]

    # Loop through blocks 1 to 6 (indices 1 to 6) in the fourth stage
    for i in range(1, 7):
        block = stage.blocks[i]  # Access the current block

        # Extract the number of input channels from the original context_module
        in_channels = block.context_module.main.qkv.conv.in_channels

        # Extract the number of output channels from the original context_module
        out_channels = block.context_module.main.proj.conv.out_channels

        # Replace the existing context_module with a new custom module
        block.context_module = nn.Sequential(
            Module(in_channels, nn.ReLU, dilation=dilation)  # Initialize custom module with required parameters
        )





In [28]:

def change_classifier(model, model_name, num_classes=4, dropout=0.5,
                     neurons1=4096, neurons2=1024, neurons3=256, neurons4=512, n_layers=2):
    """
    Change the classifier head of various vision models

    Args:
        model: The base model to modify
        model_name: Name/type of the model to determine input features
        num_classes: Number of output classes
        dropout: Dropout rate
        neurons1-4: Number of neurons in each layer
        n_layers: Number of layers in classifier (1-4)
    """
    # Define input features based on model architecture
    input_features = {
        'resnet101.a1_in1k': 2048,
        'deit3_medium_patch16_224': 512,
        'coatnet_1_rw_224.sw_in1k': 768,
        'mobilenetv3_large_100.ra_in1k': 1280,
        'vit_base_patch16_224' : 768,
        'efficientvit_l1.r224_in1k': 3072
    }

    in_features = input_features.get(model_name, 3072)  # Default to 3072 if model not found

    # Create the classifier based on number of layers
    if n_layers == 1:
        classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, neurons1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(neurons1, num_classes),
            nn.Sigmoid() if num_classes == 1 else nn.Softmax(dim=1)
        )

    elif n_layers == 2:
        classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, neurons1),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(neurons1, neurons2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(neurons2, num_classes),
            nn.Sigmoid() if num_classes == 1 else nn.Softmax(dim=1)
        )

    elif n_layers == 3:
        classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, neurons1),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(neurons1, neurons2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(neurons2, neurons3),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(neurons3, num_classes),
            nn.Sigmoid() if num_classes == 1 else nn.Softmax(dim=1)
        )

    else:  # 4 layers
        classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, neurons1),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(neurons1, neurons2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(neurons2, neurons3),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(neurons3, neurons4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(neurons4, num_classes),
            nn.Sigmoid() if num_classes == 1 else nn.Softmax(dim=1)
        )

    # Determine where to attach the classifier based on model type
    if hasattr(model, 'head'):
      if hasattr(model.head, 'classifier'):
        model.head.classifier = classifier

      elif hasattr(model.head, 'fc'):
        model.head.fc = classifier
      else:
         model.head = classifier
    elif hasattr(model, 'fc'):
      model.fc = classifier
    elif hasattr(model, 'classifier'):
      model.classifier = classifier



    else:
        raise AttributeError("Model structure not supported. Cannot find classifier or head attribute.")

    return model




In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

torch.cuda.is_available()

True

In [14]:
class_labels = {
    'Chickenpox': [1, 0, 0, 0],
    'Measles': [0, 1, 0, 0],
    'Monkeypox': [0, 0, 1, 0],
    'Normal': [0, 0, 0, 1]
}
# def acc_eval(outputs, labels, classwise=False):
#     predicted = (outputs > 0.5).float()
#     correct = (predicted == labels).float()
#     right = correct.sum().item()

#     if not classwise:
#         return right
#     else:
#         class_rights = {
#             'Other': ((1 - labels) * correct).sum().item(),
#             'Monkeypox': (labels * correct).sum().item()
#         }
#         return (right, class_rights['Other'], class_rights['Monkeypox'])

def acc_eval(outputs, labels, classwise=False):
    """
    Evaluate accuracy of model predictions.

    Args:
        outputs (torch.Tensor): Model predictions (one-hot encoded).
        labels (torch.Tensor): Ground truth labels (one-hot encoded).
        classwise (bool): If True, calculate and return class-wise accuracy.

    Returns:
        int: Total correct predictions if classwise=False.
        tuple: Total correct predictions and class-wise counts if classwise=True.
    """
    right = 0  # Counter for correct predictions
    class_rights = {class_name: 0 for class_name in class_labels.keys()}  # Initialize class-wise correct counters

    for j in range(outputs.shape[0]):  # Loop over all predictions in the batch
        max_value = torch.max(outputs[j])  # Get the maximum value in the current prediction
        outputs[j] = (outputs[j] == max_value).float()  # Convert to one-hot representation by retaining the max value index
        if torch.all(outputs[j].eq(labels[j])):  # Check if the prediction matches the ground truth
            right += 1  # Increment total correct counter
            if classwise:  # If classwise evaluation is required
                label_list = labels[j].detach().cpu().tolist()  # Convert label tensor to list for comparison
                for class_name, class_label in class_labels.items():  # Loop through each class label
                    if label_list == class_label:  # Check if the ground truth matches the current class label
                        class_rights[class_name] += 1  # Increment correct counter for the class
                        break  # Exit the loop once the class is identified
    if not classwise:  # Return total correct predictions if classwise=False
        return right
    else:  # Return total correct predictions and class-wise correct counts if classwise=True
        return (right, *class_rights.values())

In [15]:
def cal_total(labels):
    """
    Calculate the total count of labels for each class.

    Args:
        labels (torch.Tensor): Ground truth labels (one-hot encoded).

    Returns:
        tuple: Total counts for each class, in the order of class_labels keys.
    """
    # Initialize a dictionary to store the total count for each class
    class_totals = {class_name: 0 for class_name in class_labels.keys()}

    for j in range(labels.shape[0]):  # Loop over all labels in the batch
        label_list = labels[j].detach().cpu().tolist()  # Convert label tensor to a list
        for class_name, class_label in class_labels.items():  # Iterate through all class labels
            if label_list == class_label:  # Check if the label matches the current class
                class_totals[class_name] += 1  # Increment the count for the matched class
                break  # Exit the loop once the class is identified

    # Return the total counts for each class as a tuple
    return tuple(class_totals.values())


In [16]:
def norm(X_train, X_test):
    """
    Normalize training, validation, and test datasets using training set statistics.
    """
    meanx = X_train.mean()  # Calculate training set mean
    stdx = X_train.std()    # Calculate training set std

    # Normalize datasets using training set mean and std
    X_train = (X_train - meanx) / stdx
    #X_valid = (X_val - meanx) / stdx
    X_test = (X_test - meanx) / stdx

    return X_train, X_test


In [17]:
def test(model, training_loader, model_num):
    """
    Evaluate the model on the test dataset and compute accuracy, precision, recall, F1 score,
    and per-class accuracies.
    """
    right_total = 0  # Total correct predictions
    total = 0  # Total number of samples
    out = []  # List to store predicted labels
    lab = []  # List to store true labels
    class_totals = {class_name: 0 for class_name in class_labels.keys()}  # Per-class sample counts
    class_rights = {class_name: 0 for class_name in class_labels.keys()}  # Per-class correct predictions

    for i, data in enumerate(tqdm(training_loader)):  # Iterate over the training data
        inputs, labels = data  # Get inputs and labels
        total += inputs.shape[0]  # Update total samples

        outputs = model(inputs)  # Get model predictions

        # Get per-class totals and correct predictions
        class_totals_batch = cal_total(labels)
        right, *class_rights_batch = acc_eval(outputs, labels, classwise=True)

        right_total += right  # Update total correct predictions
        for i, class_name in enumerate(class_labels.keys()):  # Update per-class totals and rights
            class_totals[class_name] += class_totals_batch[i]
            class_rights[class_name] += class_rights_batch[i]

        # Convert outputs and labels to numpy arrays for evaluation
        outputs = np.array(outputs.detach().cpu(), dtype='object')
        labels = np.array(labels.detach().cpu(), dtype='object')

        out.extend(np.argmax(outputs, axis=1))  # Store predicted labels
        lab.extend(np.argmax(labels, axis=1))  # Store true labels

    accuracy = right_total / total  # Calculate overall accuracy
    class_accuracies = {class_name: class_rights[class_name] / class_totals[class_name]
                        if class_totals[class_name] > 0 else 0
                        for class_name in class_labels.keys()}  # Calculate per-class accuracies

    precision, recall, f1 = evaluate(out, lab)  # Evaluate precision, recall, and F1 score

    print("Accuracy:", accuracy)  # Print overall accuracy
    print("Total Right:", right_total)  # Print total correct predictions
    for class_name, class_accuracy in class_accuracies.items():  # Print per-class accuracy
        print(f"{class_name} Accuracy: {class_accuracy}")

    return (accuracy, precision, recall, f1, *class_accuracies.values())  # Return metrics


In [18]:
def train_one_epoch(model, epoch_index, model_num, training_loader, loss_fn, loss_fn1, w, optimizer, loss_dict_train):
    """
    Train the model for one epoch, computing losses and accuracies, and updating model weights.

    Args:
        model: The neural network model to train.
        epoch_index: The index of the current epoch.
        model_num: Identifier for the model (used for loss tracking).
        training_loader: DataLoader object for training dataset.
        loss_fn: Primary loss function used for training.
        loss_fn1: Secondary loss function used for training (combined with loss_fn).
        w: Weighting factor for combining loss_fn and loss_fn1.
        optimizer: Optimizer to update model parameters.
        loss_dict_train: Dictionary to track losses for the training process.

    Returns:
        last_loss: The average loss for the last batch in the epoch.
        overall_accuracy: The overall accuracy of the model on the training dataset.
        right_total: Total number of correct predictions in the epoch.
        class_accuracies: Per-class accuracy for each class in the dataset.
    """
    running_loss = 0.  # To track cumulative loss for the current epoch
    last_loss = 0.  # To store the average loss for the last batch
    right_total = 0  # Total correct predictions
    total = 0  # Total samples processed

    class_totals = {class_name: 0 for class_name in class_labels.keys()}  # Store count of each class in the batch
    class_rights = {class_name: 0 for class_name in class_labels.keys()}  # Store correct predictions for each class

    # Iterate through batches in the training set
    for i, data in enumerate(tqdm(training_loader)):
        inputs, labels = data  # Get input images and corresponding labels
        optimizer.zero_grad()  # Reset gradients to zero before backpropagation
        outputs = model(inputs)  # Get model predictions

        total += inputs.shape[0]  # Update the total number of samples processed
        # Compute the loss as a weighted combination of the two loss functions
        loss = (1-w) * loss_fn(outputs, labels) + w * loss_fn1(outputs, labels)
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model weights using the optimizer

        # Get per-class totals and correct predictions
        class_totals_batch = cal_total(labels)
        right, *class_rights_batch = acc_eval(outputs, labels, classwise=True)

        right_total += right  # Update the total correct predictions
        # Update per-class totals and correct predictions
        for j, class_name in enumerate(class_labels.keys()):
            class_totals[class_name] += class_totals_batch[j]
            class_rights[class_name] += class_rights_batch[j]

        running_loss += loss.item()  # Add current batch loss to running total
        # Print the average loss for every 10 batches
        if i % 10 == 9:
            last_loss = running_loss / 10  # Calculate the average loss for the last 10 batches
            print(f'  batch {i + 1} loss: {last_loss}')
            running_loss = 0.  # Reset running loss for the next set of batches

    # Calculate per-class accuracies
    print("Class totals:", class_totals)
    class_accuracies = {}
    for class_name in class_labels.keys():
        if class_totals[class_name] == 0:
            class_accuracies[class_name] = None  # No data for this class
        else:
            class_accuracies[class_name] = class_rights[class_name] / class_totals[class_name]  # Calculate class accuracy

    # Calculate overall accuracy
    overall_accuracy = right_total / total if total > 0 else None

    # Return the last loss, overall accuracy, total correct predictions, and class-wise accuracies
    return (last_loss, overall_accuracy, right_total,
            *[class_accuracies[class_name] for class_name in class_labels.keys()])


In [19]:
import numpy as np

class CAGA(nn.Module):
    """
      Attributes:
          heads: Number of attention heads in the multi-head attention mechanism.
          dim: Dimensionality of Q, K, V.
          scale: Scaling factor for the attention computation.
          head_dim: The number of channels per attention head.
          dilation: List of dilation values for dilated convolutions.
          total_layer: Total number of layers used in the module.
          get_begin: Initial convolutional layer for feature extraction.
          get_qkv: List of convolutional layers to compute queries, keys, and values.
          convert_to_headdim: Layer to combine and reshape the outputs of all dilated convolutions.
          mix: Convolutional layers to refine the features.
          proj: Final projection layer to map the concatenated features to the input shape.
          norm: Batch normalization layer to normalize the output.
      """
    def __init__(self,
            in_channels,
            activation,
            heads = 3,
            dim = 8,
            expand_ratio = 4,
            head_dim = 16,
            dilation = (1,2),
            random_seed = 82
            ):

        # Set global random seeds for reproducibility
        self._set_global_seeds(random_seed)

        super(CAGA, self).__init__()
        self.heads = heads
        self.dim = dim
        scale = dim
        self.scale = dim ** -0.5
        self.head_dim = head_dim
        self.dilation = dilation

        self.total_layer = 4

        # Reproducible layer initialization
        self.get_begin = self._init_depthwise_separable_conv(
            DepthWiseSeperableConvLayer(in_channels, self.heads * self.head_dim)
        )

        self.get_qkv = nn.ModuleList([
            nn.Sequential(
                self._init_conv(nn.Conv2d(
                    self.head_dim,
                    self.head_dim,
                    3,
                    groups=self.head_dim,
                    dilation=di
                )),
                self._init_conv(nn.Conv2d(self.head_dim, 3 * self.dim, 1, groups=1))
            )
            for di in dilation
        ])

        self.convert_to_headdim = self._init_conv(
            nn.Conv2d(len(dilation) * self.dim, self.head_dim, 1)
        )

        self.mix = nn.Sequential(
            self._init_conv(nn.Conv2d(
                self.dim,
                self.dim * 3,
                1
            )),
            nn.ReLU()
        )

        self.proj = self._init_conv(
            nn.Conv2d(self.heads * self.dim * len(self.dilation), in_channels, 1)
        )

        # Deterministic BatchNorm
        self.norm = nn.BatchNorm2d(num_features=in_channels, affine=True)
        nn.init.constant_(self.norm.weight, 1)
        nn.init.constant_(self.norm.bias, 0)

    def _set_global_seeds(self, seed):
        """Set seeds for reproducibility across libraries."""
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        np.random.seed(seed)
        random.seed(seed)

    def _init_conv(self, conv_layer):
        """Initialize convolutional layer weights deterministically."""
        nn.init.xavier_uniform_(conv_layer.weight)
        if conv_layer.bias is not None:
            nn.init.constant_(conv_layer.bias, 0)
        return conv_layer

    def _init_depthwise_separable_conv(self, conv_layer):
        """Initialize depthwise separable convolution layers."""
        # Assuming DepthWiseSeperableConvLayer has similar structure to standard conv
        for m in conv_layer.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        return conv_layer
        # self.norm = nn.LayerNorm([8, 256, 14, 14])



    def attention(self,q,k,v , shape):

        B, C, H, W = shape



        q, k, v = q.float(), k.float(), v.float()


        q = q * self.scale
        att_map = q.transpose(-2, -1) @ k


        att_map = att_map.softmax(dim=-1)


        out = v @ att_map.transpose(-2, -1)

        out = out.view(B , -1 , H , W)



        return out

    def forward(self,x):
        B, C, H, W = x.shape



            # print(op)

        x_copy = x

        all_heads  = self.get_begin(x)

        multi_layer = all_heads.split([self.head_dim]*self.heads , dim=1)

        all_heads_after_op = [[]]*self.heads



        for j in range(self.heads):

            for op in self.get_qkv:

                all_heads_after_op[j].append(op(multi_layer[j]))


        all_final = []


        for i in range(self.heads):
            out_all = []
            for j in range(len(self.dilation ) ):



                q , k , v = all_heads_after_op[i][j].split([self.dim, self.dim, self.dim], dim=1)
                shape = q.shape
                q, k, v = q.flatten(2), k.flatten(2), v.flatten(2)
                out = self.attention(q , k , v , shape)



                # print(out.shape)
                shape_ahead = all_heads_after_op[i][j+1].shape[3]

                temp = torchvision.transforms.functional.resize(out , (shape_ahead,shape_ahead))

                out = F.interpolate(out, size=( H , W ), mode='bilinear')
                out = out.view(B, self.dim, H, W)


                if j+1 != len(self.dilation ):

                    temp = self.mix(temp).clone()
                    all_heads_after_op[i][j+1] = all_heads_after_op[i][j+1] + temp

                out_all.append(out)




            out_all_one = torch.cat(out_all, dim=1)
            all_final.append(out_all_one)
            if i+1 != self.heads:
                all_heads_after_op[i+1] += self.convert_to_headdim(out_all_one)




      # we need to billinear intterpolate before append
        all_concat = torch.cat(all_final, dim=1)


        x_final = self.proj(all_concat) + x_copy # try oncat later

        return self.norm (x_final)

In [20]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.max_validation_acc = float('-inf')

    def early_stop(self, validation_acc):

        if validation_acc > self.max_validation_acc:
            self.max_validation_acc = validation_acc
            self.counter = 0
        elif validation_acc <= (self.max_validation_acc - self.min_delta):
            print("max_acc" , self.max_validation_acc)

            self.counter += 1
            if self.counter >= self.patience:
                return (True,self.counter)
        return (False,self.counter)


In [21]:
class FocalCELoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean', num_classes=4, device=None):
        """
        Focal Cross Entropy Loss

        Args:
            alpha (torch.Tensor, optional): Weight for each class. Must be of size C.
            gamma (float): Focusing parameter to reduce the relative loss for well-classified examples.
            reduction (str): Reduction method for the final loss ('none', 'mean', or 'sum').
            num_classes (int): Number of classes in the classification task.
            device (torch.device): Device on which to place the class weights and other tensors.
        """
        super(FocalCELoss, self).__init__()
        self.gamma = gamma  # Focusing parameter for down-weighting well-classified examples
        self.reduction = reduction  # Reduction method for the loss
        self.device = device if device is not None else torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Handle class weights (alpha) if provided
        if alpha is not None:
            if isinstance(alpha, (list, tuple)):
                self.alpha = torch.tensor(alpha)  # Convert list/tuple to tensor
            else:
                self.alpha = alpha  # Use alpha directly if it is already a tensor
            assert len(self.alpha) == num_classes, "Alpha size must match the number of classes"
            # Move alpha tensor to the specified device
            self.alpha = self.alpha.to(self.device)
        else:
            self.alpha = None  # If alpha is not provided, no weighting is applied

    def forward(self, inputs, targets):
        """
        Forward pass to compute the Focal Cross Entropy Loss.

        Args:
            inputs: Tensor of shape (N, C), where N is the batch size and C is the number of classes.
            targets: Tensor of shape (N,), containing class indices in the range [0, C-1].
        """
        # Ensure inputs and targets are on the same device as the class weights (alpha)
        inputs = inputs.to(self.device)
        targets = targets.to(self.device)

        # Compute the standard cross-entropy loss without reduction
        ce_loss = F.cross_entropy(inputs, targets, weight=self.alpha, reduction='none')

        # Compute the probability of the predicted class (pt)
        pt = torch.exp(-ce_loss)  # pt is the probability of the true class

        # Compute the Focal Loss by scaling the cross-entropy loss with the modulating factor
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        # Apply the specified reduction method
        if self.reduction == 'mean':
            return torch.mean(focal_loss)  # Return the mean loss
        elif self.reduction == 'sum':
            return torch.sum(focal_loss)  # Return the sum of the loss
        else:  # 'none'
            return focal_loss  # Return the loss for each sample without reduction


In [37]:
import os
import torch
import numpy as np
from datetime import datetime
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import csv
HYPERPARAMS = {
    'EPOCH': 15,
    'dropout': 0.111975,
    'neurons1': 1024,
    'neurons2': 2048,
    'neurons3': 512,
    'neurons4': 512,
    'n_layers': 2,
    'batch_size_train': 8,

    'batch_size_test': 8,
    'learning_rate': 0.0000603447274,
    'weight_decay': 0.01,

    'min_delta': 0,
    'scheduler_gamma': 0.95
}
seed = 45
def training(model_name , model_config_name, HYPERPARAMS,  X_train, y_train,  X_test, y_test, use_CAGA = False , pre_trained = True, run = 1 ):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Create necessary directories
    os.makedirs(f'./save/mymodules/run_{run}', exist_ok=True)
    os.makedirs(f'./save_data/run_{run}', exist_ok=True)

    # Initialize tracking variables
    accuracy_dict_train = []
    loss_dict_train = []
    accuracy_dict_val = []
    loss_dict_val = []
    test_metrics_history = []
    best_val_loss = float('inf')

    X_train = X_train.float()
    y_train = y_train.float()

    model_name = model_name

    csv_dir = f'/results/run_{run}'
    os.makedirs(csv_dir, exist_ok=True)

    main_csv_path = os.path.join(csv_dir, f'results_{timestamp}_{model_name}.csv')

    # Initialize main results CSV
    header = ['Model', 'Accuracy', 'Precision', 'Recall', 'F1'] + \
         [f'{class_name} Accuracy' for class_name in class_labels.keys()] + \
         list(HYPERPARAMS.keys())

    with open(main_csv_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)
    #     Monkeypox: 0
# Chickenpox: 1
# Measles: 2
# Normal: 3

    # Create model
    X_train, X_test = norm(X_train,X_test)
    model = timm.create_model(model_config_name, pretrained= pre_trained) #for ablation turn pre-trained to False
    if use_CAGA:
        replace_context_modules(model, CAGA, dilation = HYPERPARAMS['dilation']) # comment this for all other models apart for efficientViT-CAGA
    change_classifier(model, model_config_name, dropout=HYPERPARAMS['dropout'], neurons1=HYPERPARAMS['neurons1'], neurons2=HYPERPARAMS['neurons2'], neurons3=HYPERPARAMS['neurons3'], neurons4=HYPERPARAMS['neurons4'], n_layers=HYPERPARAMS['n_layers'])

    print(model)
    model.to(device)



    # Create data loaders
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    generator = torch.Generator()
    generator.manual_seed(seed)

    training_loader = DataLoader(
        list(zip(X_train.to(device), y_train.to(device))),
        batch_size=HYPERPARAMS['batch_size_train'],
        shuffle=True,
        generator=generator,
        worker_init_fn=lambda worker_id: torch.manual_seed(seed + worker_id)
    )

    # Test loader doesn't need shuffling but we'll set the seed for consistency
    test_loader = DataLoader(
        list(zip(X_test.to(device), y_test.to(device))),
        batch_size=HYPERPARAMS['batch_size_test'],
        shuffle=False,
        generator=generator
    )

    #alpha = torch.tensor([1.25, 0.61, 0.81, 0.59]).to(device)
    alpha = torch.tensor([1.29, 0.66, 1.15, 0.81]).to(device)
    #alpha = torch.tensor([1.25 ,1.35,1.15, 0.51]).to(device)
    # Loss functions and optimizer setup
    #loss_fn = FocalCELoss( alpha = alpha, gamma=2.0, num_classes=4)
    loss_fn = torch.nn.BCELoss()
    loss_fn1 = torch.nn.CrossEntropyLoss(weight = alpha)
    w = 0
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=HYPERPARAMS['learning_rate'], weight_decay=HYPERPARAMS['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=HYPERPARAMS['scheduler_gamma'])
    #early_stopper = EarlyStopper(patience=HYPERPARAMS['patience'], min_delta=HYPERPARAMS['min_delta'])

    def evaluate_on_test():
        model.eval()
        with torch.no_grad():
            acc, precision, recall, f1, *class_accuracies_test = test(model, test_loader, 0)
            print('\n##########################################################')
            print(f'Intermediate Test Results:')
            print(f'Accuracy test: {acc:.4f}')
            print(f'Precision test: {precision:.4f}')
            print(f'Recall test: {recall:.4f}')
            print(f'F1 test: {f1:.4f}')
            print('---------->classwise test<-----------')
            for i, class_name in enumerate(class_labels.keys()):
                print(f'Accuracy {class_name} test: {class_accuracies_test[i]:.4f}')
            print('##########################################################\n')

            return {
                'epoch': epoch,  # Corrected indentation here
                'accuracy': acc,
                'precision': precision,
                'recall': recall,
                'f1': f1,
                'class_accuracies': class_accuracies_test
            }

    # Rest of the function remains the same...
    # Training loop
    for epoch in range(HYPERPARAMS['EPOCH']):
        print(f"Learning rate: {optimizer.param_groups[0]['lr']}")
        print(f'EPOCH {epoch }:')

        # Training phase
        model.train(True)
        avg_loss_train, train_accuracy, right, *class_accuracies_train = train_one_epoch(
            model, epoch, 0, training_loader, loss_fn, loss_fn1, w, optimizer, loss_dict_train
        )
        accuracy_dict_train.append(train_accuracy)
        loss_dict_train.append(avg_loss_train)

        print(f"Average training loss: {avg_loss_train}")
        print(f'Right train {right}')
        print(f'Accuracy train {train_accuracy}')

        print('---------->classwise train<-----------')
        for i, class_name in enumerate(class_labels.keys()):
            print(f'Accuracy {class_name} train {class_accuracies_train[i]}')

        # Evaluate on test set every 1 epoch
        #if (epoch + 1) % 1 == 0:
         #   test_metrics = evaluate_on_test()
          #  test_metrics_history.append(test_metrics)

        scheduler.step()

    # Final evaluation
    final_test_metrics = evaluate_on_test()
    test_metrics_history.append(final_test_metrics)

    row = [model_name, final_test_metrics['accuracy'], final_test_metrics['precision'], final_test_metrics['recall'], final_test_metrics['f1']] + \
                    final_test_metrics['class_accuracies'] + \
                    list(HYPERPARAMS.values())

    with open(main_csv_path, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row)

    return {
        'accuracy': final_test_metrics['accuracy'],
        'precision': final_test_metrics['precision'],
        'recall': final_test_metrics['recall'],
        'f1': final_test_metrics['f1'],
        'train_accuracies': accuracy_dict_train,
        'train_losses': loss_dict_train,
        'class_accuracies': final_test_metrics['class_accuracies'],
        'test_history': test_metrics_history
    }

